# <center> TLS 1.2 Deep Dive
## <center> SYSE 549: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 
### <center> Instructor: Dr. Jeremy Daily<br>
### <center> Content Author: Jerry Duggan

## Learning Objectives
By the end of this module, you will:
* Understand how to create and verify a self-signed certificate;
* Understand how to initiate a TLS v1.2 session with an HTTP server;
* Understand how to access components of a TLS certificate with a client application, and how to verify the certificate;
* Understand how to use Wireshark to examine the steps for establishing a TLS session.

## Quick Review
Certificates are used to authenticate clients and servers. They are the basis for securing websites and other communications over the Internet. They are defined in RFC 5280: https://datatracker.ietf.org/doc/html/rfc5280.html.

The primary reason for certificates and a Certificate Authority (CA) is to overcome the person-in-the-middle (PITM) attack as shown below.

![Person in the Middle](PITM.png)

![Certificates](CA.png)

## Generating and Using a Certificate
This tutorial follows along with the approach found at https://cryptography.io/en/latest/x509/.

We'll start out by creating a certificate and putting together a certificate signing request.


### Load dependencies
This was written with crytography.io at version 40.

In [1]:
%pip install --upgrade --user cryptography

Requirement already up-to-date: cryptography in c:\users\jdaily\appdata\roaming\python\python38\site-packages (40.0.1)
Note: you may need to restart the kernel to use updated packages.


### Generate private key & calculate it's associated public key

In [2]:
from cryptography.hazmat.primitives.asymmetric import rsa
privateKey = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
publicKey = privateKey.public_key()
print(publicKey)

### Create an x509 object with our pertinent information

In [3]:
from cryptography import x509
issuer = x509.Name([
        x509.NameAttribute(x509.NameOID.COUNTRY_NAME,           u"US"),
        x509.NameAttribute(x509.NameOID.STATE_OR_PROVINCE_NAME, u"Colorado"),
        x509.NameAttribute(x509.NameOID.LOCALITY_NAME,          u"Fort Collins"),
        x509.NameAttribute(x509.NameOID.ORGANIZATION_NAME,      u"Colorado State University"),
        x509.NameAttribute(x509.NameOID.COMMON_NAME,            u"fakesite.edu")
    ])
issuer

<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>

### This is a self-signed certificate, so in the case the issuer is also the subject

In [4]:
subject = issuer

### Create the certificate builder...

In [5]:
from datetime import datetime, timedelta
rawCert = (
        x509.CertificateBuilder()
            .subject_name(subject)
            .issuer_name(issuer)
            .public_key(publicKey)
            .serial_number(x509.random_serial_number())
            .not_valid_before(datetime.utcnow())
            .not_valid_after(datetime.utcnow() + timedelta(days=10))
            .add_extension(x509.SubjectAlternativeName([x509.DNSName(u"localhost")]), critical=False)
            )
rawCert

### And sign it with our private key to make it a real certificate

In [6]:
from cryptography.hazmat.primitives import hashes
signedCert = rawCert.sign(privateKey, hashes.SHA256())
signedCert

<Certificate(subject=<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>, ...)>

In [7]:
from cryptography.hazmat.primitives import serialization
signedCert_PEM = signedCert.public_bytes(serialization.Encoding.PEM)
print(signedCert_PEM)

b'-----BEGIN CERTIFICATE-----\nMIIDijCCAnKgAwIBAgIUEk5EXmAXYhUY9laLXDMbqRjDSnwwDQYJKoZIhvcNAQEL\nBQAwcjELMAkGA1UEBhMCVVMxETAPBgNVBAgMCENvbG9yYWRvMRUwEwYDVQQHDAxG\nb3J0IENvbGxpbnMxIjAgBgNVBAoMGUNvbG9yYWRvIFN0YXRlIFVuaXZlcnNpdHkx\nFTATBgNVBAMMDGZha2VzaXRlLmVkdTAeFw0yMzA0MDUyMTQ3MDZaFw0yMzA0MTUy\nMTQ3MDZaMHIxCzAJBgNVBAYTAlVTMREwDwYDVQQIDAhDb2xvcmFkbzEVMBMGA1UE\nBwwMRm9ydCBDb2xsaW5zMSIwIAYDVQQKDBlDb2xvcmFkbyBTdGF0ZSBVbml2ZXJz\naXR5MRUwEwYDVQQDDAxmYWtlc2l0ZS5lZHUwggEiMA0GCSqGSIb3DQEBAQUAA4IB\nDwAwggEKAoIBAQC1veT2k8xjKzqJxslQk94mOaPrpcCsE7T71Zaz79TgyMoGhC9J\nw5UQ/4uOlZc12G7PvY2y0jmgnVhmhg8mo/IxTN/b0CuJRDM5sARG045qWQd1qwXb\nctv/tVSG2d4bSpcx47syyHhTUBo8fcZt1dC1c69VGzmksKlwciINcHkLqD07rikR\nmt0RLXHgCiOTRN8Eu421Y4DN3Pd7Q+AzNFS8cfR6f/9A1f+J31knQ5UMEYyKW30j\niDto200QxswWvZB/lIU7+856gFyj7ygaCeNr7Om+yCXNz+QzqsEtfsIo2AHBEmd5\nfrH0PXru43+zEkDLp5D6YfSNxrnFCHiX0pV7AgMBAAGjGDAWMBQGA1UdEQQNMAuC\nCWxvY2FsaG9zdDANBgkqhkiG9w0BAQsFAAOCAQEAc+KvuTzDaV/hd0sWy4XmplcG\ncKueI/y2kgNxwP+UDNq3VcaPWvUSYwFfDl+YiNw+1Ergs

### Pretend the public key & certificate were transmitted...

In [8]:
# Instead of sending it, we;ll just save it to disk
with open('cert.pem','wb') as f:
    f.write(signedCert_PEM)

### Pretend the client receives the certificate...

In [9]:
# Instead of receiveing it, we;ll just open it from disk
with open('cert.pem','rb') as f:
    received_pem = f.read()
print(received_pem)

b'-----BEGIN CERTIFICATE-----\nMIIDijCCAnKgAwIBAgIUEk5EXmAXYhUY9laLXDMbqRjDSnwwDQYJKoZIhvcNAQEL\nBQAwcjELMAkGA1UEBhMCVVMxETAPBgNVBAgMCENvbG9yYWRvMRUwEwYDVQQHDAxG\nb3J0IENvbGxpbnMxIjAgBgNVBAoMGUNvbG9yYWRvIFN0YXRlIFVuaXZlcnNpdHkx\nFTATBgNVBAMMDGZha2VzaXRlLmVkdTAeFw0yMzA0MDUyMTQ3MDZaFw0yMzA0MTUy\nMTQ3MDZaMHIxCzAJBgNVBAYTAlVTMREwDwYDVQQIDAhDb2xvcmFkbzEVMBMGA1UE\nBwwMRm9ydCBDb2xsaW5zMSIwIAYDVQQKDBlDb2xvcmFkbyBTdGF0ZSBVbml2ZXJz\naXR5MRUwEwYDVQQDDAxmYWtlc2l0ZS5lZHUwggEiMA0GCSqGSIb3DQEBAQUAA4IB\nDwAwggEKAoIBAQC1veT2k8xjKzqJxslQk94mOaPrpcCsE7T71Zaz79TgyMoGhC9J\nw5UQ/4uOlZc12G7PvY2y0jmgnVhmhg8mo/IxTN/b0CuJRDM5sARG045qWQd1qwXb\nctv/tVSG2d4bSpcx47syyHhTUBo8fcZt1dC1c69VGzmksKlwciINcHkLqD07rikR\nmt0RLXHgCiOTRN8Eu421Y4DN3Pd7Q+AzNFS8cfR6f/9A1f+J31knQ5UMEYyKW30j\niDto200QxswWvZB/lIU7+856gFyj7ygaCeNr7Om+yCXNz+QzqsEtfsIo2AHBEmd5\nfrH0PXru43+zEkDLp5D6YfSNxrnFCHiX0pV7AgMBAAGjGDAWMBQGA1UdEQQNMAuC\nCWxvY2FsaG9zdDANBgkqhkiG9w0BAQsFAAOCAQEAc+KvuTzDaV/hd0sWy4XmplcG\ncKueI/y2kgNxwP+UDNq3VcaPWvUSYwFfDl+YiNw+1Ergs

In [10]:
# Let's load the PEM file into a certificate object
received_cert = x509.load_pem_x509_certificate(received_pem)
received_cert

<Certificate(subject=<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>, ...)>

In [11]:
#This should match what we just loaded.
received_cert.public_bytes(serialization.Encoding.PEM)

b'-----BEGIN CERTIFICATE-----\nMIIDijCCAnKgAwIBAgIUEk5EXmAXYhUY9laLXDMbqRjDSnwwDQYJKoZIhvcNAQEL\nBQAwcjELMAkGA1UEBhMCVVMxETAPBgNVBAgMCENvbG9yYWRvMRUwEwYDVQQHDAxG\nb3J0IENvbGxpbnMxIjAgBgNVBAoMGUNvbG9yYWRvIFN0YXRlIFVuaXZlcnNpdHkx\nFTATBgNVBAMMDGZha2VzaXRlLmVkdTAeFw0yMzA0MDUyMTQ3MDZaFw0yMzA0MTUy\nMTQ3MDZaMHIxCzAJBgNVBAYTAlVTMREwDwYDVQQIDAhDb2xvcmFkbzEVMBMGA1UE\nBwwMRm9ydCBDb2xsaW5zMSIwIAYDVQQKDBlDb2xvcmFkbyBTdGF0ZSBVbml2ZXJz\naXR5MRUwEwYDVQQDDAxmYWtlc2l0ZS5lZHUwggEiMA0GCSqGSIb3DQEBAQUAA4IB\nDwAwggEKAoIBAQC1veT2k8xjKzqJxslQk94mOaPrpcCsE7T71Zaz79TgyMoGhC9J\nw5UQ/4uOlZc12G7PvY2y0jmgnVhmhg8mo/IxTN/b0CuJRDM5sARG045qWQd1qwXb\nctv/tVSG2d4bSpcx47syyHhTUBo8fcZt1dC1c69VGzmksKlwciINcHkLqD07rikR\nmt0RLXHgCiOTRN8Eu421Y4DN3Pd7Q+AzNFS8cfR6f/9A1f+J31knQ5UMEYyKW30j\niDto200QxswWvZB/lIU7+856gFyj7ygaCeNr7Om+yCXNz+QzqsEtfsIo2AHBEmd5\nfrH0PXru43+zEkDLp5D6YfSNxrnFCHiX0pV7AgMBAAGjGDAWMBQGA1UdEQQNMAuC\nCWxvY2FsaG9zdDANBgkqhkiG9w0BAQsFAAOCAQEAc+KvuTzDaV/hd0sWy4XmplcG\ncKueI/y2kgNxwP+UDNq3VcaPWvUSYwFfDl+YiNw+1Ergs

In [12]:
# So, what does the PEM say?
received_cert.version

<Version.v3: 2>

In [13]:
received_cert.fingerprint(hashes.SHA256())

b'>\xc1\xbb:\x98\xef\xaa\xf4:]\x00h\x14\x87s\\\xb8\x88*\xffK\xd2q\x04&\x9f}X\xf6\xbaU\xfa'

In [14]:
received_cert.serial_number

104507247750027650076980151987079926071664265852

In [15]:
cert_pub_key = received_cert.public_key()
cert_pub_key

In [16]:
received_cert.not_valid_before

datetime.datetime(2023, 4, 5, 21, 47, 6)

In [17]:
received_cert.not_valid_after

datetime.datetime(2023, 4, 15, 21, 47, 6)

In [18]:
received_cert.issuer

<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>

In [19]:
received_cert.subject

<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>

In [20]:
received_cert.signature_hash_algorithm

In [21]:
received_cert.signature_algorithm_oid

<ObjectIdentifier(oid=1.2.840.113549.1.1.11, name=sha256WithRSAEncryption)>

In [22]:
received_cert.signature

b's\xe2\xaf\xb9<\xc3i_\xe1wK\x16\xcb\x85\xe6\xa6W\x06p\xab\x9e#\xfc\xb6\x92\x03q\xc0\xff\x94\x0c\xda\xb7U\xc6\x8fZ\xf5\x12c\x01_\x0e_\x98\x88\xdc>\xd4J\xe0\xb0\x19\x8a^d\x9d\ts^\x90\xde%\xcc\x02\xa2\xe0\xb8\xbbl=\xdbB-\xc3\xedFe\xedD\xe9\x03\x93k\x84O\xe0\xaa\xcf\xfe\xfd\xa0\xe5\xee\xf4\xeb\xb0\x15\xd1\x0b\xadg\'\x90F\x11$\xbe\x82\x87O9\xdcC\x88\xbf\x10\xc58\xc2\xd5\xe6o\xd9\x15\x91Pg1g\xfa\x94"|\xb7L\x1d\xa9\xd7\xdb\xd5r\xbe\xa2\x03\xa7h@~\xa0\xd5\xd4+u`6u\xdb\x03\xff\xa3|y\x8f%n\x8c\xf6\x80U\x9aOX\r_dA\x90\x06\xf3r6Qn\x959bj\xe6\xbe\x98\xaf\xffm\xea\xc8\x8e\xb0\x0f\xc5\xde\x00.\xd2p%w3.\x93\xcb\x1bD\xdcss\xd8e\xe3\x80\xec^\x02\xf7|\xa2\x1c\xa9\x06\xfe\xc9\x0c\x93O/\x1b\xfb\xdc\xde\x13\xe5\xe4\xe7\n\xb5\xb6\xf4\xc8K\x87\x98eF\xe3\xbc\x04'

In [23]:
for ext in received_cert.extensions:
    print(ext)

<Extension(oid=<ObjectIdentifier(oid=2.5.29.17, name=subjectAltName)>, critical=False, value=<SubjectAlternativeName(<GeneralNames([<DNSName(value='localhost')>])>)>)>


In [24]:
#The DER encoded bytes payload (as defined by RFC 5280) that is hashed and then
#signed by the private key of the certificate’s issuer.
received_cert.tbs_certificate_bytes

b'0\x82\x02r\xa0\x03\x02\x01\x02\x02\x14\x12ND^`\x17b\x15\x18\xf6V\x8b\\3\x1b\xa9\x18\xc3J|0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x0b\x05\x000r1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x110\x0f\x06\x03U\x04\x08\x0c\x08Colorado1\x150\x13\x06\x03U\x04\x07\x0c\x0cFort Collins1"0 \x06\x03U\x04\n\x0c\x19Colorado State University1\x150\x13\x06\x03U\x04\x03\x0c\x0cfakesite.edu0\x1e\x17\r230405214706Z\x17\r230415214706Z0r1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x110\x0f\x06\x03U\x04\x08\x0c\x08Colorado1\x150\x13\x06\x03U\x04\x07\x0c\x0cFort Collins1"0 \x06\x03U\x04\n\x0c\x19Colorado State University1\x150\x13\x06\x03U\x04\x03\x0c\x0cfakesite.edu0\x82\x01"0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x01\x05\x00\x03\x82\x01\x0f\x000\x82\x01\n\x02\x82\x01\x01\x00\xb5\xbd\xe4\xf6\x93\xccc+:\x89\xc6\xc9P\x93\xde&9\xa3\xeb\xa5\xc0\xac\x13\xb4\xfb\xd5\x96\xb3\xef\xd4\xe0\xc8\xca\x06\x84/I\xc3\x95\x10\xff\x8b\x8e\x95\x975\xd8n\xcf\xbd\x8d\xb2\xd29\xa0\x9dXf\x86\x0f&\xa3\xf21L\xdf\xdb\xd0+\x89D39\xb0\x04F\xd3\x8ejY\x07u\xab\x0

### And now the "client" verifies  that the certificate has not been altered.

In [25]:
from cryptography.hazmat.primitives.asymmetric import padding
cert_pub_key.verify(
        received_cert.signature,
        received_cert.tbs_certificate_bytes,
        padding.PKCS1v15(),
        received_cert.signature_hash_algorithm
    )
print("Valid signature")

Valid signature


In [26]:
#We get an exception if the certificate or signature are altered.
from cryptography.exceptions import InvalidSignature
try:
    cert_pub_key.verify(
        received_cert.signature,
        b'1' + received_cert.tbs_certificate_bytes[1:], # change just the first byte to get manipulated data
        padding.PKCS1v15(),
        received_cert.signature_hash_algorithm
    )
    print("Valid signature")
except InvalidSignature as e:
    print(repr(e))

InvalidSignature()


In [27]:
datetime.utcnow() < received_cert.not_valid_after

True

In [28]:
datetime.utcnow() > received_cert.not_valid_before

True

In [29]:
# Check expiration:
if (datetime.utcnow() < received_cert.not_valid_after and
    datetime.utcnow() > received_cert.not_valid_before):
    print("The certificate has not expired")
else:
    print("The certificate is outside the allowed times.")

The certificate has not expired


## TLS Deep Dive

In [30]:
import requests

In [31]:
r = requests.get('https://github.com')
r

<Response [200]>

https://github.com/certifi/python-certifi

In [32]:
import certifi
certifi.where()

'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'

In [33]:
with open(certifi.where(),'rb') as f:
    root_CAs = f.read()
root_CAs [:1000]

b'\n# Issuer: CN=GlobalSign Root CA O=GlobalSign nv-sa OU=Root CA\n# Subject: CN=GlobalSign Root CA O=GlobalSign nv-sa OU=Root CA\n# Label: "GlobalSign Root CA"\n# Serial: 4835703278459707669005204\n# MD5 Fingerprint: 3e:45:52:15:09:51:92:e1:b7:5d:37:9f:b1:87:29:8a\n# SHA1 Fingerprint: b1:bc:96:8b:d4:f4:9d:62:2a:a8:9a:81:f2:15:01:52:a4:1d:82:9c\n# SHA256 Fingerprint: eb:d4:10:40:e4:bb:3e:c7:42:c9:e3:81:d3:1e:f2:a4:1a:48:b6:68:5c:96:e7:ce:f3:c1:df:6c:d4:33:1c:99\n-----BEGIN CERTIFICATE-----\nMIIDdTCCAl2gAwIBAgILBAAAAAABFUtaw5QwDQYJKoZIhvcNAQEFBQAwVzELMAkG\nA1UEBhMCQkUxGTAXBgNVBAoTEEdsb2JhbFNpZ24gbnYtc2ExEDAOBgNVBAsTB1Jv\nb3QgQ0ExGzAZBgNVBAMTEkdsb2JhbFNpZ24gUm9vdCBDQTAeFw05ODA5MDExMjAw\nMDBaFw0yODAxMjgxMjAwMDBaMFcxCzAJBgNVBAYTAkJFMRkwFwYDVQQKExBHbG9i\nYWxTaWduIG52LXNhMRAwDgYDVQQLEwdSb290IENBMRswGQYDVQQDExJHbG9iYWxT\naWduIFJvb3QgQ0EwggEiMA0GCSqGSIb3DQEBAQUAA4IBDwAwggEKAoIBAQDaDuaZ\njc6j40+Kfvvxi4Mla+pIH/EqsLmVEQS98GPR4mdmzxzdzxtIK+6NiY6arymAZavp\nxy0Sy6scTHAHoT0KMM0VjU/43dSMUBUc71DuxC73/O

In [34]:
len(root_CAs)

282394

In [35]:
root_certs = x509.load_pem_x509_certificates(root_CAs)
root_certs

C:\Users\jdaily\AppData\Roaming\Python\Python38\site-packages\cryptography\x509\base.py:576: CryptographyDeprecationWarning: Parsed a negative serial number, which is disallowed by RFC 5280.
  return rust_x509.load_pem_x509_certificates(data)


[<Certificate(subject=<Name(C=BE,O=GlobalSign nv-sa,OU=Root CA,CN=GlobalSign Root CA)>, ...)>,
 <Certificate(subject=<Name(OU=GlobalSign Root CA - R2,O=GlobalSign,CN=GlobalSign)>, ...)>,
 <Certificate(subject=<Name(O=Entrust.net,OU=www.entrust.net/CPS_2048 incorp. by ref. (limits liab.),OU=(c) 1999 Entrust.net Limited,CN=Entrust.net Certification Authority (2048))>, ...)>,
 <Certificate(subject=<Name(C=IE,O=Baltimore,OU=CyberTrust,CN=Baltimore CyberTrust Root)>, ...)>,
 <Certificate(subject=<Name(C=US,O=Entrust\, Inc.,OU=www.entrust.net/CPS is incorporated by reference,OU=(c) 2006 Entrust\, Inc.,CN=Entrust Root Certification Authority)>, ...)>,
 <Certificate(subject=<Name(C=US,O=GeoTrust Inc.,CN=GeoTrust Global CA)>, ...)>,
 <Certificate(subject=<Name(C=US,O=GeoTrust Inc.,CN=GeoTrust Universal CA)>, ...)>,
 <Certificate(subject=<Name(C=US,O=GeoTrust Inc.,CN=GeoTrust Universal CA 2)>, ...)>,
 <Certificate(subject=<Name(C=GB,ST=Greater Manchester,L=Salford,O=Comodo CA Limited,CN=AAA Cert

In [36]:
for rootCA in root_certs:
    print(rootCA.issuer)

<Name(C=BE,O=GlobalSign nv-sa,OU=Root CA,CN=GlobalSign Root CA)>
<Name(OU=GlobalSign Root CA - R2,O=GlobalSign,CN=GlobalSign)>
<Name(O=Entrust.net,OU=www.entrust.net/CPS_2048 incorp. by ref. (limits liab.),OU=(c) 1999 Entrust.net Limited,CN=Entrust.net Certification Authority (2048))>
<Name(C=IE,O=Baltimore,OU=CyberTrust,CN=Baltimore CyberTrust Root)>
<Name(C=US,O=Entrust\, Inc.,OU=www.entrust.net/CPS is incorporated by reference,OU=(c) 2006 Entrust\, Inc.,CN=Entrust Root Certification Authority)>
<Name(C=US,O=GeoTrust Inc.,CN=GeoTrust Global CA)>
<Name(C=US,O=GeoTrust Inc.,CN=GeoTrust Universal CA)>
<Name(C=US,O=GeoTrust Inc.,CN=GeoTrust Universal CA 2)>
<Name(C=GB,ST=Greater Manchester,L=Salford,O=Comodo CA Limited,CN=AAA Certificate Services)>
<Name(C=BM,O=QuoVadis Limited,OU=Root Certification Authority,CN=QuoVadis Root Certification Authority)>
<Name(C=BM,O=QuoVadis Limited,CN=QuoVadis Root CA 2)>
<Name(C=BM,O=QuoVadis Limited,CN=QuoVadis Root CA 3)>
<Name(C=JP,O=SECOM Trust.net,O

These are trusted certificates that can be stored locally and used to verify signatures. Let's see how this is done.

### Establish a Socket Connection

We have seen how to establish a socket connection before.  The folks who implement TLS in Python have gone to a lot of trouble to make it transparent to use a TLS-enabled socket.  The problem is that the normal ways of doing so "hide" a lot of the details, and we explicitly want access to those details.  So we have to go to a bit of extra trouble to get this to work.

### Load Prereqs

In [38]:
import socket
import ssl

### Parameters to Use

In [77]:
HOSTNAME = 'cybertruck1.engr.colostate.edu'
DEST_IPADDR = '129.82.20.206'
PORT = 443

To prepare for this section:
* Open Wireshark
* Set a capture filter to 'host 129.82.20.206 and port 443'
* Set a display filter to 'tls'
![WiresharkSetupforTLS.png](WiresharkSetupforTLS.png)

In [86]:
ssl.get_server_certificate((HOSTNAME,PORT),ssl.PROTOCOL_TLSv1_2)

'-----BEGIN CERTIFICATE-----\nMIIGxjCCBa6gAwIBAgIQO+gndbZGxQAAibXK0Y+GqDANBgkqhkiG9w0BAQsFADB2\nMQswCQYDVQQGEwJVUzELMAkGA1UECBMCTUkxEjAQBgNVBAcTCUFubiBBcmJvcjES\nMBAGA1UEChMJSW50ZXJuZXQyMREwDwYDVQQLEwhJbkNvbW1vbjEfMB0GA1UEAxMW\nSW5Db21tb24gUlNBIFNlcnZlciBDQTAeFw0yMzAzMjAwMDAwMDBaFw0yNDA0MTky\nMzU5NTlaMG0xCzAJBgNVBAYTAlVTMREwDwYDVQQIEwhDb2xvcmFkbzEiMCAGA1UE\nChMZQ29sb3JhZG8gU3RhdGUgVW5pdmVyc2l0eTEnMCUGA1UEAxMeY3liZXJ0cnVj\nazEuZW5nci5jb2xvc3RhdGUuZWR1MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIB\nCgKCAQEA07ihTYI9/kRbUPujwkNa6e6p2YW5rTICJOIixnbes0PN/bbPOcQrrUF1\ntY5h5WFBTmMiSN4ZJBWShFii0a9LmNi5ciqwHtak2/8lfckoFMDNQpS3N2iKvyc4\nuMuNcyGlcn+z/OOYFza555uzeempvO7TNPggOfytnSw4PtQPwey6n3FdMx1Y4vB6\nGJJ+QZV/SYx5N/AbUFKr8sqEW3MxgvzcpVfH/AbQkVwLPBGBWDlci8jz1iloCN4d\nDWDlY/CeJ0vSnc0/CLB9arcXMo9Qcqlf4iPnqtkLb3zCFH63vo0GPnmHg6WcjHNK\nWH7xsuRFdk0Egj9Q5fwkM09D+wrBTwIDAQABo4IDVzCCA1MwHwYDVR0jBBgwFoAU\nHgWjd49sluJbh0umtIascQAM5zgwHQYDVR0OBBYEFGcMjNFBcvxs3GIsb3nqbbeo\n6Ju5MA4GA1UdDwEB/wQEAwIFoDAMBgNVHRMBAf8EAjAAMB

This produced a recording in wireshark.
![TLS1.2WiresharkCapture.png](TLS1.2WiresharkCapture.png)

In [93]:
server_pem = ssl.get_server_certificate((HOSTNAME,PORT),ssl.PROTOCOL_TLS)
server_pem

'-----BEGIN CERTIFICATE-----\nMIIGxjCCBa6gAwIBAgIQO+gndbZGxQAAibXK0Y+GqDANBgkqhkiG9w0BAQsFADB2\nMQswCQYDVQQGEwJVUzELMAkGA1UECBMCTUkxEjAQBgNVBAcTCUFubiBBcmJvcjES\nMBAGA1UEChMJSW50ZXJuZXQyMREwDwYDVQQLEwhJbkNvbW1vbjEfMB0GA1UEAxMW\nSW5Db21tb24gUlNBIFNlcnZlciBDQTAeFw0yMzAzMjAwMDAwMDBaFw0yNDA0MTky\nMzU5NTlaMG0xCzAJBgNVBAYTAlVTMREwDwYDVQQIEwhDb2xvcmFkbzEiMCAGA1UE\nChMZQ29sb3JhZG8gU3RhdGUgVW5pdmVyc2l0eTEnMCUGA1UEAxMeY3liZXJ0cnVj\nazEuZW5nci5jb2xvc3RhdGUuZWR1MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIB\nCgKCAQEA07ihTYI9/kRbUPujwkNa6e6p2YW5rTICJOIixnbes0PN/bbPOcQrrUF1\ntY5h5WFBTmMiSN4ZJBWShFii0a9LmNi5ciqwHtak2/8lfckoFMDNQpS3N2iKvyc4\nuMuNcyGlcn+z/OOYFza555uzeempvO7TNPggOfytnSw4PtQPwey6n3FdMx1Y4vB6\nGJJ+QZV/SYx5N/AbUFKr8sqEW3MxgvzcpVfH/AbQkVwLPBGBWDlci8jz1iloCN4d\nDWDlY/CeJ0vSnc0/CLB9arcXMo9Qcqlf4iPnqtkLb3zCFH63vo0GPnmHg6WcjHNK\nWH7xsuRFdk0Egj9Q5fwkM09D+wrBTwIDAQABo4IDVzCCA1MwHwYDVR0jBBgwFoAU\nHgWjd49sluJbh0umtIascQAM5zgwHQYDVR0OBBYEFGcMjNFBcvxs3GIsb3nqbbeo\n6Ju5MA4GA1UdDwEB/wQEAwIFoDAMBgNVHRMBAf8EAjAAMB

In [94]:
#What does this say?
server_certs = x509.load_pem_x509_certificates(server_pem.encode('ascii'))
for inter_cert in server_certs:
    print(inter_cert)

<Certificate(subject=<Name(C=US,ST=Colorado,O=Colorado State University,CN=cybertruck1.engr.colostate.edu)>, ...)>


In [180]:
context = ssl.create_default_context()
context.load_verify_locations(cafile=certifi.where())
context.check_hostname

True

In [181]:
context.options

<Options.OP_NO_SSLv3|OP_CIPHER_SERVER_PREFERENCE|OP_ENABLE_MIDDLEBOX_COMPAT|OP_NO_COMPRESSION|OP_ALL: -2108555180>

In [178]:
for c in context.get_ca_certs():
    for item in c['issuer']:
        if item[0][0] == 'commonName':
            print(item[0][1])

ISRG Root X1
Certigna
TeliaSonera Root CA v1
Izenpe.com
Sonera Class2 CA
Amazon Root CA 1
Amazon Root CA 2
Amazon Root CA 3
Amazon Root CA 4
Cybertrust Global Root
Atos TrustedRoot 2011
UCA Global G2 Root
DST Root CA X3
AffirmTrust Premium
GeoTrust Global CA
ACCVRAIZ1
AffirmTrust Commercial
AffirmTrust Networking
QuoVadis Root CA 2
QuoVadis Root CA 3
SwissSign Gold CA - G2
AffirmTrust Premium ECC
GeoTrust Universal CA
SwissSign Silver CA - G2
UCA Extended Validation Root
Hongkong Post Root CA 1
GeoTrust Universal CA 2
GTS Root R1
GTS Root R2
GTS Root R3
GTS Root R4
QuoVadis Root CA 1 G3
QuoVadis Root CA 2 G3
QuoVadis Root CA 3 G3
SecureTrust CA
IdenTrust Commercial Root CA 1
Secure Global CA
DigiCert Global Root CA
GlobalSign
GlobalSign
GlobalSign
D-TRUST Root Class 3 CA 2 2009
IdenTrust Public Sector Root CA 1
DigiCert Global Root CA
DigiCert Global Root CA
Buypass Class 2 Root CA
Buypass Class 3 Root CA
DigiCert Global Root CA
Baltimore CyberTrust Root
ISRG Root X1
D-TRUST Root Class

### Create the socket connection

The call to 'do_handshake' does all the work required establish the connection.

In [222]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    with context.wrap_socket(sock, server_hostname=HOSTNAME) as conn:
        conn.settimeout(5)
        conn.connect((DEST_IPADDR, PORT))
        conn.setblocking(1)
        conn.do_handshake()
        print(conn)
        cert = conn.getpeercert()
        cert_bin = conn.getpeercert(binary_form=True)

<ssl.SSLSocket fd=1480, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.205.5.72', 62348), raddr=('129.82.20.206', 443)>


In [225]:
cert

{'subject': ((('countryName', 'US'),),
  (('stateOrProvinceName', 'Colorado'),),
  (('organizationName', 'Colorado State University'),),
  (('commonName', 'cybertruck1.engr.colostate.edu'),)),
 'issuer': ((('countryName', 'US'),),
  (('stateOrProvinceName', 'MI'),),
  (('localityName', 'Ann Arbor'),),
  (('organizationName', 'Internet2'),),
  (('organizationalUnitName', 'InCommon'),),
  (('commonName', 'InCommon RSA Server CA'),)),
 'version': 3,
 'serialNumber': '3BE82775B646C5000089B5CAD18F86A8',
 'notBefore': 'Mar 20 00:00:00 2023 GMT',
 'notAfter': 'Apr 19 23:59:59 2024 GMT',
 'subjectAltName': (('DNS', 'cybertruck1.engr.colostate.edu'),),
 'OCSP': ('http://ocsp.usertrust.com',),
 'caIssuers': ('http://crt.usertrust.com/InCommonRSAServerCA_2.crt',),
 'crlDistributionPoints': ('http://crl.incommon-rsa.org/InCommonRSAServerCA.crl',)}

In [227]:
cert['caIssuers'][0]

'http://crt.usertrust.com/InCommonRSAServerCA_2.crt'

In [224]:
cert_bin[:300]

b'0\x82\x06\xc60\x82\x05\xae\xa0\x03\x02\x01\x02\x02\x10;\xe8\'u\xb6F\xc5\x00\x00\x89\xb5\xca\xd1\x8f\x86\xa80\r\x06\t*\x86H\x86\xf7\r\x01\x01\x0b\x05\x000v1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x0b0\t\x06\x03U\x04\x08\x13\x02MI1\x120\x10\x06\x03U\x04\x07\x13\tAnn Arbor1\x120\x10\x06\x03U\x04\n\x13\tInternet21\x110\x0f\x06\x03U\x04\x0b\x13\x08InCommon1\x1f0\x1d\x06\x03U\x04\x03\x13\x16InCommon RSA Server CA0\x1e\x17\r230320000000Z\x17\r240419235959Z0m1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x110\x0f\x06\x03U\x04\x08\x13\x08Colorado1"0 \x06\x03U\x04\n\x13\x19Colorado State University1\'0%\x06\x03U\x04\x03\x13\x1ecybertruck1.engr.colo'

In [155]:
server_cert = x509.load_der_x509_certificate(cert_bin)
server_cert

<Certificate(subject=<Name(C=US,ST=Colorado,O=Colorado State University,CN=cybertruck1.engr.colostate.edu)>, ...)>

In [189]:

for attribute in server_cert.issuer.get_attributes_for_oid(x509.NameOID.COMMON_NAME):
    print(attribute.value)
issuer_name = attribute.value

InCommon RSA Server CA


In [195]:
for e in server_cert.extensions:
    print(e)

<Extension(oid=<ObjectIdentifier(oid=2.5.29.35, name=authorityKeyIdentifier)>, critical=False, value=<AuthorityKeyIdentifier(key_identifier=b'\x1e\x05\xa3w\x8fl\x96\xe2[\x87K\xa6\xb4\x86\xacq\x00\x0c\xe78', authority_cert_issuer=None, authority_cert_serial_number=None)>)>
<Extension(oid=<ObjectIdentifier(oid=2.5.29.14, name=subjectKeyIdentifier)>, critical=False, value=<SubjectKeyIdentifier(digest=b'g\x0c\x8c\xd1Ar\xfcl\xdcb,oy\xeam\xb7\xa8\xe8\x9b\xb9')>)>
<Extension(oid=<ObjectIdentifier(oid=2.5.29.15, name=keyUsage)>, critical=True, value=<KeyUsage(digital_signature=True, content_commitment=False, key_encipherment=True, data_encipherment=False, key_agreement=False, key_cert_sign=False, crl_sign=False, encipher_only=False, decipher_only=False)>)>
<Extension(oid=<ObjectIdentifier(oid=2.5.29.19, name=basicConstraints)>, critical=True, value=<BasicConstraints(ca=False, path_length=None)>)>
<Extension(oid=<ObjectIdentifier(oid=2.5.29.37, name=extendedKeyUsage)>, critical=False, value=<Ex

In [197]:
issuers_constant = x509.oid.AuthorityInformationAccessOID.CA_ISSUERS
issuers_constant

<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.2, name=caIssuers)>

In [203]:
access_constant = x509.ExtensionOID.AUTHORITY_INFORMATION_ACCESS
access_constant

<ObjectIdentifier(oid=1.3.6.1.5.5.7.1.1, name=authorityInfoAccess)>

In [205]:
server_cert.extensions.get_extension_for_oid(access_constant)

<Extension(oid=<ObjectIdentifier(oid=1.3.6.1.5.5.7.1.1, name=authorityInfoAccess)>, critical=False, value=<AuthorityInformationAccess([<AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.2, name=caIssuers)>, access_location=<UniformResourceIdentifier(value='http://crt.usertrust.com/InCommonRSAServerCA_2.crt')>)>, <AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.1, name=OCSP)>, access_location=<UniformResourceIdentifier(value='http://ocsp.usertrust.com')>)>])>)>

In [211]:
server_cert.extensions.get_extension_for_oid(access_constant).value

<AuthorityInformationAccess([<AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.2, name=caIssuers)>, access_location=<UniformResourceIdentifier(value='http://crt.usertrust.com/InCommonRSAServerCA_2.crt')>)>, <AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.1, name=OCSP)>, access_location=<UniformResourceIdentifier(value='http://ocsp.usertrust.com')>)>])>

In [213]:
for a in server_cert.extensions.get_extension_for_oid(access_constant).value:
    print(a)

<AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.2, name=caIssuers)>, access_location=<UniformResourceIdentifier(value='http://crt.usertrust.com/InCommonRSAServerCA_2.crt')>)>
<AccessDescription(access_method=<ObjectIdentifier(oid=1.3.6.1.5.5.7.48.1, name=OCSP)>, access_location=<UniformResourceIdentifier(value='http://ocsp.usertrust.com')>)>


In [217]:
caIssuers = server_cert.extensions.get_extension_for_oid(access_constant).value[0].access_location.value
caIssuers

'http://crt.usertrust.com/InCommonRSAServerCA_2.crt'

In [218]:
r = requests.get(caIssuers)
r.content

b'0\x82\x05\xf90\x82\x03\xe1\xa0\x03\x02\x01\x02\x02\x10G \xd0\xfa\x85F\x1a~\x17\xa1d\x02\x91\x84ct0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x0c\x05\x000\x81\x881\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x130\x11\x06\x03U\x04\x08\x13\nNew Jersey1\x140\x12\x06\x03U\x04\x07\x13\x0bJersey City1\x1e0\x1c\x06\x03U\x04\n\x13\x15The USERTRUST Network1.0,\x06\x03U\x04\x03\x13%USERTrust RSA Certification Authority0\x1e\x17\r141006000000Z\x17\r241005235959Z0v1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x0b0\t\x06\x03U\x04\x08\x13\x02MI1\x120\x10\x06\x03U\x04\x07\x13\tAnn Arbor1\x120\x10\x06\x03U\x04\n\x13\tInternet21\x110\x0f\x06\x03U\x04\x0b\x13\x08InCommon1\x1f0\x1d\x06\x03U\x04\x03\x13\x16InCommon RSA Server CA0\x82\x01"0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x01\x05\x00\x03\x82\x01\x0f\x000\x82\x01\n\x02\x82\x01\x01\x00\x9c\x1b\xf1\xbb/\x7fc\x18\x15QQT\x0f\x9e\xc5NM\x10X\xfa0\x9b\x17)\x90\xe63\x0c\xac\x13S|T\x91\xb4\xea\xd8n\x9b\x89m\xbb3>\x8f\xd2\r\xa6\xe9\xf9\xba\xe9\r\x0c\x1a\x9e\xb2\x8e\xc9p.\xef\x1e\x05}\x95\xeb-\

In [146]:
#generate a list of known roots
root_names = []
for c in root_certs:
    try:
        root_names.append(c.issuer.get_attributes_for_oid(x509.NameOID.COMMON_NAME)[-1].value)
    except IndexError:
        pass
root_names

['GlobalSign Root CA',
 'GlobalSign',
 'Entrust.net Certification Authority (2048)',
 'Baltimore CyberTrust Root',
 'Entrust Root Certification Authority',
 'GeoTrust Global CA',
 'GeoTrust Universal CA',
 'GeoTrust Universal CA 2',
 'AAA Certificate Services',
 'QuoVadis Root Certification Authority',
 'QuoVadis Root CA 2',
 'QuoVadis Root CA 3',
 'Sonera Class2 CA',
 'XRamp Global Certification Authority',
 'DigiCert Assured ID Root CA',
 'DigiCert Global Root CA',
 'DigiCert High Assurance EV Root CA',
 'DST Root CA X3',
 'SwissSign Gold CA - G2',
 'SwissSign Silver CA - G2',
 'GeoTrust Primary Certification Authority',
 'thawte Primary Root CA',
 'VeriSign Class 3 Public Primary Certification Authority - G5',
 'SecureTrust CA',
 'Secure Global CA',
 'COMODO Certification Authority',
 'Network Solutions Certificate Authority',
 'COMODO ECC Certification Authority',
 'OISTE WISeKey Global Root GA CA',
 'Certigna',
 'Cybertrust Global Root',
 'GeoTrust Primary Certification Authority 

In [158]:
issuer_name in root_names

False

In [102]:
r=requests.get(cert['caIssuers'][0])
r

<Response [200]>

In [109]:
#See the content of the certificate in DER format
r.content

b'0\x82\x05\xf90\x82\x03\xe1\xa0\x03\x02\x01\x02\x02\x10G \xd0\xfa\x85F\x1a~\x17\xa1d\x02\x91\x84ct0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x0c\x05\x000\x81\x881\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x130\x11\x06\x03U\x04\x08\x13\nNew Jersey1\x140\x12\x06\x03U\x04\x07\x13\x0bJersey City1\x1e0\x1c\x06\x03U\x04\n\x13\x15The USERTRUST Network1.0,\x06\x03U\x04\x03\x13%USERTrust RSA Certification Authority0\x1e\x17\r141006000000Z\x17\r241005235959Z0v1\x0b0\t\x06\x03U\x04\x06\x13\x02US1\x0b0\t\x06\x03U\x04\x08\x13\x02MI1\x120\x10\x06\x03U\x04\x07\x13\tAnn Arbor1\x120\x10\x06\x03U\x04\n\x13\tInternet21\x110\x0f\x06\x03U\x04\x0b\x13\x08InCommon1\x1f0\x1d\x06\x03U\x04\x03\x13\x16InCommon RSA Server CA0\x82\x01"0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x01\x05\x00\x03\x82\x01\x0f\x000\x82\x01\n\x02\x82\x01\x01\x00\x9c\x1b\xf1\xbb/\x7fc\x18\x15QQT\x0f\x9e\xc5NM\x10X\xfa0\x9b\x17)\x90\xe63\x0c\xac\x13S|T\x91\xb4\xea\xd8n\x9b\x89m\xbb3>\x8f\xd2\r\xa6\xe9\xf9\xba\xe9\r\x0c\x1a\x9e\xb2\x8e\xc9p.\xef\x1e\x05}\x95\xeb-\

In [114]:
intermediate_cert = x509.load_der_x509_certificate(r.content)
intermediate_cert

<Certificate(subject=<Name(C=US,ST=MI,L=Ann Arbor,O=Internet2,OU=InCommon,CN=InCommon RSA Server CA)>, ...)>

In [124]:
intermediate_cert.subject

<Name(C=US,ST=MI,L=Ann Arbor,O=Internet2,OU=InCommon,CN=InCommon RSA Server CA)>

In [125]:
intermediate_cert.issuer

<Name(C=US,ST=New Jersey,L=Jersey City,O=The USERTRUST Network,CN=USERTrust RSA Certification Authority)>

In [145]:
for attribute in intermediate_cert.issuer.get_attributes_for_oid(x509.NameOID.COMMON_NAME):
    print(attribute.value)
issuer_name = attribute.value

USERTrust RSA Certification Authority


In [148]:
#Check to see if the issuer of the intermetidate certificate is in a root of trust.
issuer_name in root_names

True

### Set up an SSL context, which will allow us to control and query a lot of the parameters for the exchange

In [50]:
cert['issuer']


((('countryName', 'US'),),
 (('stateOrProvinceName', 'MI'),),
 (('localityName', 'Ann Arbor'),),
 (('organizationName', 'Internet2'),),
 (('organizationalUnitName', 'InCommon'),),
 (('commonName', 'InCommon RSA Server CA'),))

In [51]:
for rootCA in root_certs:
    print(rootCA.issuer.org)

AttributeError: 'Name' object has no attribute 'org'

Bring up your Wireshark window now

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    conn = SSL.Connection(context, socket=s)
    conn.settimeout(5)
    conn.connect((DEST_IPADDR, PORT))
    conn.setblocking(1)
    conn.do_handshake()
    peerCert = conn.get_peer_certificate()  # Get the certificate from the packet
    certChain = conn.get_peer_cert_chain()  # Get the certificates of interest from the local trust store


### Extract the certs we care about 

TBH, we are using two different cryptography libraries here.  I'm using OpenSSL only because it has a call to extract the certificate chain from the message.  I'd rather use the cryptography library (which we've looked at before) to do the heavy lifting...

In [ ]:
CACert = certChain[-1].to_cryptography()
pktCert = peerCert.to_cryptography()

### Repeat the steps from the sequence diagram...

![Certificates](CA.png)

#### Verify the certificate with the public key from our local cert store

In [ ]:
#
# Verify certificate w/ CA public key
#

try:
    CACert.public_key().verify(
            pktCert.signature,
            pktCert.tbs_certificate_bytes,
            padding.PKCS1v15(),
            pktCert.signature_hash_algorithm
        )
    print("Valid signature")
except InvalidSignature as e:
    print("Invalid signature")

#### Verify the hostname & certificate validity times

In [ ]:
# todo

#### Verify the hashes

In [ ]:
# todo